# **Docking**
## 1. Vina Setup

*   Place the AutoDock Vina executable (v1.2.7; https://github.com/ccsb-scripps/AutoDock-Vina/releases) in the working folder (`contents`).
*   Import ligand and receptor `.pdbqt` files from the preprocessing step.

## 2. Docking Grid Centres
Docking grid centres from cleaned PyMOL structures, with water and ions removed.

The following table lists the coordinates for each receptor, representing the centre of the docking box that covers the ATP-binding site:


| Receptor Type       | PDB ID | Centre (Å)               |
|--------------------|--------|--------------------------|
| **Wildtype**       | 4WKQ   | [1.257, 194.291, 20.788] |
|                    | 4G5J   | [50.680, 1.374, -21.036] |
|                    | 8F1X   | [-0.520, 51.458, -19.642] |
| **L858R**          | 2ITZ   | [-51.712, -1.102, -22.327] |
|                    | 6JWL   | [-49.947, -0.565, -18.541] |
|                    | 2ITT   | [-51.800, -0.285, -19.735] |
| **T790M**          | 6JX0   | [55.262, 43.520, 126.579] |
|                    | 4G5P   | [-10.922, 19.113, 31.245] |
|                    | 5GMP   | [0.396, -50.840, 126.002] |
| **Exon20 Insertion** | 9GL8  | [-1.187, 17.249, -10.016] |
|                    | 4LRM   | [48.884, 361.406, 32.215] |
|                    | 9GC6   | [32.816, -24.714, 47.255] |


## 3. Organise Ligands and Receptors
Ligands and receptors are organised into separate folders (`ligand_pdbqt` and `proteins_pdbqt`), and a `docking_results` folder is created to store outputs.

The script moves each `.pdbqt` file to its respective folder and flags any missing files. This ensures the workflow remains structured and allows all ligand–receptor combinations to be processed efficiently.

In [ ]:
import os
import subprocess
import shutil

In [ ]:
# Make Vina executable runnable
vina_executable = "./vina_1.2.7_linux_x86_64"
os.chmod(vina_executable, 0o755)

In [ ]:
# Create directories for ligands, receptors, and results
os.makedirs("ligand_pdbqt", exist_ok=True)
os.makedirs("proteins_pdbqt", exist_ok=True)
os.makedirs("docking_results", exist_ok=True)

In [ ]:
# Define ligands and receptors
ligand_files = [
    "Gefitinib.pdbqt",
    "Erlotinib.pdbqt",
    "Afatinib.pdbqt",
    "Mobocertinib.pdbqt",
    "Osimertinib.pdbqt",
    "Aspirin.pdbqt",
    "Caffeine.pdbqt",
    "Ibuprofen.pdbqt"
]

receptor_files = [
    # Wildtype
    "EGFR_wt_4WKQ_nowater.pdbqt",
    "EGFR_wt_4G5J_nowater.pdbqt",
    "EGFR_wt_8F1X_nowater.pdbqt",
    # L858R
    "EGFR_L858R_2ITZ_nowater.pdbqt",
    "EGFR_L858R_6JWL_nowater.pdbqt",
    "EGFR_L858R_2ITT_nowater.pdbqt",
    # T790M
    "EGFR_T790M_6JX0_nowater.pdbqt",
    "EGFR_T790M_4G5P_nowater.pdbqt",
    "EGFR_T790M_5GMP_nowater.pdbqt",
    # Exon20 Insertion
    "EGFR_exon20_9GL8_nowater.pdbqt",
    "EGFR_exon20_4LRM_nowater.pdbqt",
    "EGFR_exon20_9GC6_nowater.pdbqt"
]

In [ ]:
# Move ligand files
for f in ligand_files:
    src = f
    dst = os.path.join("ligand_pdbqt", f)
    if os.path.exists(src):
        shutil.move(src, dst)
    else:
        print(f"Ligand not found: {f}")

# Move receptor files
for f in receptor_files:
    src = f
    dst = os.path.join("proteins_pdbqt", f)
    if os.path.exists(src):
        shutil.move(src, dst)
    else:
        print(f"Receptor not found: {f}")

## 4. Define Docking Boxes
Each receptor is assigned a cubic docking box of 20 Å per side, centred on coordinates determined from the cleaned PyMOL structures. This box fully encompasses the ATP-binding site while allowing ligands sufficient freedom to adopt flexible conformations.

The centres and dimensions are stored in a dictionary for automated use in the docking script, maintaining consistency across all docking runs.

In [ ]:
docking_boxes = {
    # Wildtype
    "EGFR_wt_4WKQ_nowater.pdbqt": {"center": [1.257, 194.291, 20.788], "size": [20, 20, 20]},
    "EGFR_wt_4G5J_nowater.pdbqt": {"center": [50.680, 1.374, -21.036], "size": [20, 20, 20]},
    "EGFR_wt_8F1X_nowater.pdbqt": {"center": [-0.520, 51.458, -19.642], "size": [20, 20, 20]},

    # L858R
    "EGFR_L858R_2ITZ_nowater.pdbqt": {"center": [-51.712, -1.102, -22.327], "size": [20, 20, 20]},
    "EGFR_L858R_6JWL_nowater.pdbqt": {"center": [-49.947, -0.565, -18.541], "size": [20, 20, 20]},
    "EGFR_L858R_2ITT_nowater.pdbqt": {"center": [-51.800, -0.285, -19.735], "size": [20, 20, 20]},

    # T790M
    "EGFR_T790M_6JX0_nowater.pdbqt": {"center": [55.262, 43.520, 126.579], "size": [20, 20, 20]},
    "EGFR_T790M_4G5P_nowater.pdbqt": {"center": [-10.922, 19.113, 31.245], "size": [20, 20, 20]},
    "EGFR_T790M_5GMP_nowater.pdbqt": {"center": [0.396, -50.840, 126.002], "size": [20, 20, 20]},

    # Exon20 Insertion
    "EGFR_exon20_9GL8_nowater.pdbqt": {"center": [-1.187, 17.249, -10.016], "size": [20, 20, 20]},
    "EGFR_exon20_4LRM_nowater.pdbqt": {"center": [48.884, 361.406, 32.215], "size": [20, 20, 20]},
    "EGFR_exon20_9GC6_nowater.pdbqt": {"center": [32.816, -24.714, 47.255], "size": [20, 20, 20]}
}


## 5. Run Docking with Vina
Docking is performed for each ligand–receptor pair using AutoDock Vina. Exhaustiveness is set to 8, and only the top-scoring pose is retained per ligand.

Outputs include the docked `.pdbqt` files and `.log` files containing predicted binding affinities and docking details.

Progress messages and errors are printed to the console, allowing immediate identification of any failed runs and ensuring reproducibility.

In [ ]:
for ligand in ligand_files:
    ligand_path = os.path.join("ligand_pdbqt", ligand)
    if not os.path.exists(ligand_path):
        print(f"Ligand file missing: {ligand_path}")
        continue

    for receptor in receptor_files:
        receptor_path = os.path.join("proteins_pdbqt", receptor)
        if not os.path.exists(receptor_path):
            print(f"Receptor file missing: {receptor_path}")
            continue

        box = docking_boxes[receptor]
        cx, cy, cz = box["center"]
        sx, sy, sz = box["size"]

        out_file = os.path.join("docking_results", f"{ligand.split('.')[0]}_{receptor.split('.')[0]}.pdbqt")
        log_file = os.path.join("docking_results", f"{ligand.split('.')[0]}_{receptor.split('.')[0]}.log")

        cmd = [
            vina_executable,
            "--receptor", receptor_path,
            "--ligand", ligand_path,
            "--center_x", str(cx),
            "--center_y", str(cy),
            "--center_z", str(cz),
            "--size_x", str(sx),
            "--size_y", str(sy),
            "--size_z", str(sz),
            "--out", out_file,
            "--exhaustiveness", "8",
            "--num_modes", "1"
        ]

        print(f"[DOCKING] {ligand} → {receptor}")
        with open(log_file, "w") as f:
            try:
                subprocess.run(cmd, stdout=f, stderr=subprocess.STDOUT, check=True)
                print(f"Saved {out_file}")
            except subprocess.CalledProcessError as e:
                print(f"Docking failed for {ligand} → {receptor}: {e}")